# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')

In [ ]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [ ]:
df.head(15)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [ ]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
df.isna().mean().sort_values(ascending = False)

is_ultra    0.0
mb_used     0.0
messages    0.0
minutes     0.0
calls       0.0
dtype: float64

### Вывод:

В этом пункте проекта было проведено ознакомление с данными: пропуски не были обнаружены, данные нужных типов, предобработка не требуется.

## Разбейте данные на выборки

Разобьем данные на обучающую, валидационную и тестовые выборки в соотношении 3:1:1 :

In [ ]:
train_df, valid_and_test_df = train_test_split(df, test_size=0.40, random_state=244466666)
valid_df, test_df = train_test_split(valid_and_test_df, test_size=0.50, random_state=244466666)

print('Кол-во объектов в обучающей выборке:', train_df.shape[0])
print('Кол-во объектов в валидационной выборке:', valid_df.shape[0])
print('Кол-во объектов в тестовой выборке:', test_df.shape[0])

Кол-во объектов в обучающей выборке: 1928
Кол-во объектов в валидационной выборке: 643
Кол-во объектов в тестовой выборке: 643


Создадим отдельные переменные для целевого и остальных признаков для каждой выборки:

In [ ]:
features_train = train_df.drop(['is_ultra'], axis=1)
target_train = train_df['is_ultra']
features_valid = valid_df.drop(['is_ultra'], axis=1)
target_valid = valid_df['is_ultra']
features_test = test_df.drop(['is_ultra'], axis=1)
target_test = test_df['is_ultra']

### Вывод:

Данные были разбиты в соотношении 3:1:1.

* Обучающая выборка - train_df  
* Валидационная выборка - valid_df  
* Тестовая выборка - test_df  

## Исследуйте модели

Рассмотрим для данной задачи 3 модели: дерево решений, случайный лес и логистическую регрессию  

### Дерево решений

In [ ]:
%%time

best_depth = 0
best_criterion = None
best_features = 0
best_accuracy = 0
best_decision_tree_model = None

for depth in tqdm(range(1,11)):
    for crit in ['gini', 'entropy']:
        for max_features in [None, 'sqrt', 'log2']:
            decision_tree_model = DecisionTreeClassifier(random_state=244466666, max_depth=depth, 
                                                         criterion=crit, max_features=max_features)
            decision_tree_model.fit(features_train, target_train)
            predictions_valid = decision_tree_model.predict(features_valid)
            accuracy = accuracy_score(target_valid, predictions_valid)
            if accuracy > best_accuracy:
                best_depth = depth
                best_criterion = crit
                best_features = max_features
                best_accuracy = accuracy
                best_decision_tree_model = decision_tree_model
        
print(' Лучшая глубина дерева:', best_depth, '\n', 'Лучший критерий:',  best_criterion, '\n',
      'Признак, по которому ищется лучшее разбиение:', best_features, '\n', 'Лучшая точность:', best_accuracy)  

100%|██████████| 10/10 [00:00<00:00, 24.00it/s]

 Лучшая глубина дерева: 8 
 Лучший критерий: entropy 
 Признак, по которому ищется лучшее разбиение: sqrt 
 Лучшая точность: 0.8009331259720062
CPU times: user 374 ms, sys: 2 ms, total: 376 ms
Wall time: 420 ms


Было обучено 1000 моделей с различными значениями глубины от 1 до 1000, лучшее значение 8.  
Для того, чтобы не перегружать проект, представлен цикл со значениями глубины от 1 до 10.

In [ ]:
best_decision_tree_model

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=8,
                       max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=244466666, splitter='best')

### Cлучайный лес

In [ ]:
%%time

best_est = 0
best_criterion = None
best_features = 0
best_depth = 0
best_accuracy = 0
best_random_forest_model = None

for est in tqdm(range(15, 21)):
    for crit in ['gini', 'entropy']:
        for max_features in [None, 'sqrt', 'log2']:
            for depth in range(30,41):
                random_forest_model = RandomForestClassifier(random_state=244466666, n_estimators=est, criterion=crit, max_features=max_features, max_depth=depth) # обучите модель с заданным количеством деревьев
                random_forest_model.fit(features_train, target_train) # обучите модель на тренировочной выборке
                predictions_valid = random_forest_model.predict(features_valid)
                accuracy = accuracy_score(target_valid, predictions_valid)
                if accuracy > best_accuracy:
                    best_est = est
                    best_criterion = crit
                    best_features = max_features
                    best_depth = depth
                    best_accuracy = accuracy
                    best_random_forest_model = random_forest_model
                    
print(' Лучшее кол-во деревьев в лесу:', best_est, '\n', 'Лучший критерий:',  best_criterion, '\n',
      'Признак, по которому ищется лучшее разбиение:', best_features, '\n',
      'Лучшая глубина дерева:', best_depth, '\n', 'Лучшая точность:', best_accuracy)

100%|██████████| 6/6 [00:50<00:00,  8.44s/it]

 Лучшее кол-во деревьев в лесу: 17 
 Лучший критерий: entropy 
 Признак, по которому ищется лучшее разбиение: None 
 Лучшая глубина дерева: 31 
 Лучшая точность: 0.8118195956454122
CPU times: user 49.4 s, sys: 140 ms, total: 49.6 s
Wall time: 50.6 s


Было обучено 60000 моделей с различными значениями количества деревьев от 1 до 100, и глубины от 1 до 100. Лучшее значение количества деревьев - 17, глубины - 31.  
Для того, чтобы не перегружать проект, представлен цикл со значениями количества деревьев от 15 до 20, и глубины от 30 до 40.

In [ ]:
best_random_forest_model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=31, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=17,
                       n_jobs=None, oob_score=False, random_state=244466666,
                       verbose=0, warm_start=False)

### Логистическая регрессия

In [ ]:
%%time

best_solver = None
best_accuracy = 0
best_max_iter = 0
best_accuracy = 0
best_logistic_regression_model = None

for solver in tqdm(['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']):
    for max_iter in range(50,101):
        logistic_regression_model = LogisticRegression(random_state=244466666, solver=solver, max_iter=max_iter)
        logistic_regression_model.fit(features_train, target_train) # обучите модель на тренировочной выборке
        predictions_valid = logistic_regression_model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predictions_valid)
        if accuracy > best_accuracy:
            best_max_iter = max_iter
            best_solver = solver
            best_accuracy = accuracy
            best_logistic_regression_model = logistic_regression_model

print(' Лучший solver:', best_solver, '\n', 'Лучшее количество итераций:', best_max_iter, '\n',
      'Лучшая точность:', best_accuracy)  

100%|██████████| 5/5 [00:11<00:00,  2.36s/it]

 Лучший solver: lbfgs 
 Лучшее количество итераций: 87 
 Лучшая точность: 0.7465007776049767
CPU times: user 11.5 s, sys: 7.83 ms, total: 11.5 s
Wall time: 11.8 s


Было обучено 5000 моделей с различными значениями количества итераций от 1 до 1000 лучшее значение 87.  
Для того, чтобы не перегружать проект, представлен цикл со значениями количества итераций от 50 до 100.

In [ ]:
best_logistic_regression_model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=87,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=244466666, solver='lbfgs', tol=0.0001,
                   verbose=0, warm_start=False)

## Проверьте модель на тестовой выборке

### Дерево решений 

In [ ]:
%%time

test_predictions = best_decision_tree_model.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
print('Точность решающего дерева на тестовой выборке',accuracy)

Точность решающего дерева на тестовой выборке 0.7838258164852255
CPU times: user 2.7 ms, sys: 25 µs, total: 2.73 ms
Wall time: 2.21 ms


### Случайный лес 

In [ ]:
%%time

test_predictions = best_random_forest_model.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
print('Точность случайного леса на тестовой выборке',accuracy)

Точность случайного леса на тестовой выборке 0.7900466562986003
CPU times: user 2.06 ms, sys: 7.98 ms, total: 10 ms
Wall time: 8.43 ms


### Логистическая регрессия

In [ ]:
%%time

test_predictions = best_logistic_regression_model.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
print('Точность логистической регресии на тестовой выборке',accuracy)

Точность логистической регресии на тестовой выборке 0.7387247278382582
CPU times: user 1.18 ms, sys: 3.97 ms, total: 5.16 ms
Wall time: 46.8 ms


### Вывод:

Точность моделей в порядке убывания:
1. Случайный лес
2. Дерево решений
3. Логистическая регрессия

Скорость моделей в порядке убывания:
1. Логистическая регрессия
2. Дерево решений
3. Случайный лес

## (бонус) Проверьте модели на адекватность

In [ ]:
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

Если модель всегда будет предсказывать тариф Смарт, то ее точность будет 69.35%, точность модели случаного леса - 79%.   
Проверка на адекватность пройдена успешно.

## Вывод

В ходе работы над данным проектом были было проведено исследование с разными типами моделей и выбрана модель с максимально большим значением accuracy, ей оказалась моджель случайного леса с такими параметрами:

* Кол-во деревьев в лесу - 17 
* Критерий - entropy 
* Признак, по которому ищется лучшее разбиение - None 
* Глубина дерева - 31 

Точность модели при таких параметрах на тестовой выборке равна 0.7900466562986003

Также модель успешно прошла проверку на адекватность.